In [8]:
from calitp_data_analysis.tables import tbls
from siuba import *
import pandas as pd
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/ntd/"

pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None) 

In [16]:
# using siuba to read in data from warehouse.
ntd_info = (tbls.mart_ntd.dim_annual_ntd_agency_information() 
            >> filter(_._is_current == True,
                     _.year == 2022,
                     _.region == 9) 
            >> collect()
           )

# need to find a way to filter  ntd service to California agencies only. 
ntd_service = (tbls.mart_ntd.dim_annual_ntd_agency_service()
               >> group_by(_.ntd_id)
               >> distinct(_.agency_name)
               >> collect()
            )#can i add pd filters after this?

# reading in NTD ID crosswalk from GCS
crosswalk = pd.read_csv(
        f"{GCS_FILE_PATH}ntd_id_rtpa_crosswalk.csv", 
        dtype = {"NTD ID": "str"}
    ).rename(columns={"NTD ID": "ntd_id"})

In [17]:
display(
    ntd_info.shape,
    ntd_service.shape,
    crosswalk.shape
)

(322, 46)

(2259, 2)

(117, 6)

In [19]:
ntd_service.head()

,ntd_id,agency_name
0,00001,King County Department of Metro Transit
1,00002,Spokane Transit Authority
2,00003,Pierce County Transportation Benefit Area Auth...
3,00004,Confederated Tribes of the Colville Indian Res...
4,00005,City of Everett
